In [1]:
# Sending commands from parent to child
from H5Gizmos import Html, serve, do, get

async def task():
    greeting = Html("<h1>Hello</h1>")
    await greeting.show()

    # Get some values:
    innerHeight = await get(greeting.window.innerHeight)
    print ("The inner height is", innerHeight)
    height = await get(greeting.element.height())
    print ("The element height is", height)

    # Do some actions
    do(greeting.window.console.log("testing testing", [1, 2, 3]))
    do(greeting.element.text("Goodbye!"))

await task()

The inner height is 16
The element height is 38.88888931274414


In [2]:
from H5Gizmos import Html, get, do
greeting = Html("<div><p>Paragraph 1</p> <p>Paragraph 2</p><div>")
await greeting.show()

In [3]:
greeting.element

_CACHE_['jQueryComponent_1651257029705_6'][V(L('element'))]

In [4]:
greeting.element.children().length

_CACHE_['jQueryComponent_1651257029705_6'][V(L('element'))][V(L('children'))]()[V(L('length'))]

In [5]:
await get(greeting.element.children().length)

3

In [6]:
await get(greeting.element.children()[0].innerText)

'Paragraph 1'

In [7]:
await get(greeting.window.clientInformation.vendor)

'Google Inc.'

In [8]:
await get(greeting.document.location.hostname)


'127.0.0.1'

In [9]:
await get(greeting.jQuery().jquery)

'3.1.1'

In [10]:
from H5Gizmos import Html, do

div = Html("<div>The increment function</div>")
await div.show()
do(div.jQuery("<pre>lambda(x): x+1</pre>").appendTo(div.element))

In [11]:
test = await get(greeting.element.no_such_function(3))

JavascriptEvalException: js error: 'Error: get no_such_function from [object Object] not truthy: undefined'

In [12]:
from H5Gizmos import Html, name, get
greeting = Html("<h4>Hello</h4>")
await greeting.show()

location = name("location", greeting.document.location)

In [13]:
await get(location.toString())

'http://127.0.0.1:8676/gizmo/http/MGR_1651257244020_11/index.html'

In [15]:
import numpy as np
from H5Gizmos import Html

greeting = Html("<h3>Hello</h3>")
await greeting.show()

def callable_example(*arguments):
    print("callback called with arguments: ", arguments)

structure = {
    "none": None,
    "number": 3.1,
    "string": "Bohemian Rhapsody",
    "bool": False,
    "dictionary": {"lie": "falsehood"},
    "list": [1,2,"three"],
    "bytearray": bytearray([0,5,211,6]),
    "callable": callable_example,
    "1d numeric array": np.arange(10),
    "2d numeric array": np.arange(10).reshape(2,5),
}
do(greeting.window.console.log(structure))

In [16]:
from H5Gizmos.python.gz_components import JS_COLLECTION_NAME_MAP

In [17]:
JS_COLLECTION_NAME_MAP

{numpy.int8: 'Int8Array',
 numpy.uint8: 'Uint8Array',
 numpy.int16: 'Uint16Array',
 numpy.int32: 'Int32Array',
 numpy.uint32: 'Uint32Array',
 numpy.float32: 'Float32Array',
 numpy.float64: 'Float64Array',
 numpy.int64: 'BigInt64Array',
 numpy.uint64: 'BigUint64Array',
 dtype('int8'): 'Int8Array',
 dtype('uint8'): 'Uint8Array',
 dtype('int16'): 'Uint16Array',
 dtype('int32'): 'Int32Array',
 dtype('uint32'): 'Uint32Array',
 dtype('float32'): 'Float32Array',
 dtype('float64'): 'Float64Array',
 dtype('int64'): 'BigInt64Array',
 dtype('uint64'): 'BigUint64Array'}

In [18]:
import numpy as np
C = np.zeros((3,), dtype=np.complex)
do(greeting.window.console.log(C))

CantConvertValue: No conversion for: <class 'complex'>

In [19]:
from H5Gizmos import Html, name, get
greeting = Html("<h4>Hello</h4>")
await greeting.show()

greeting.js_init("""

    element.send_value = function() {
        return {
            "null": null,
            "number": 3.1,
            "string": "Bohemian Rhapsody",
            "bool": false,
            "dictionary": {"lie": "falsehood"},
            "list": [1,2,"three"],
            "Uint8Array": new Uint8Array([0, 5, 211, 6]),
            "callable": element.send_value,
        }
    };
""")

In [20]:
js_value = await get(greeting.element.send_value())
js_value

{'null': None,
 'number': 3.1,
 'string': 'Bohemian Rhapsody',
 'bool': False,
 'dictionary': {'lie': 'falsehood'},
 'list': [1, 2, 'three'],
 'Uint8Array': '0005d306',
 'callable': {}}

In [21]:
greeting.js_init("""
    var loop = {};
    loop.name = "loop";
    loop.reference = loop;
    element.loop = loop;
""")

In [22]:
await get(greeting.element.loop, to_depth=2)

{'name': 'loop', 'reference': {'name': 'loop', 'reference': None}}

In [23]:
await get(greeting.element.loop, to_depth=4)

{'name': 'loop',
 'reference': {'name': 'loop',
  'reference': {'name': 'loop',
   'reference': {'name': 'loop', 'reference': None}}}}

In [24]:
w = await get(greeting.window, to_depth=2)
print(len(repr(w)))

36325


In [25]:
from H5Gizmos import Html, name, get
greeting = Html("<h4>Hello</h4>")
await greeting.show()

greeting.js_init("""

    element.resolves_in_10_seconds = function() {
        var result = new H5Gizmos.DeferredValue();
        // Resolve the value to 42 after waiting 10 seconds.
        setTimeout( (function() { result.resolve(42);}), 10 * 1000 );
        return result;
    };

""")

In [26]:
await get(greeting.element.resolves_in_10_seconds(), timeout=13)

42

In [27]:
await get(greeting.element.resolves_in_10_seconds(), timeout=5)

FutureTimeout: Timeout expired: 5

In [28]:
from H5Gizmos import Html, get
greeting = Html("<h1>Hello</h1>")
await greeting.show()
txt = greeting.add("Welcome!")

properties = "font-style font-weight font-family font-size".split()

greeting.js_init(
    """
        element.get_font_properties = function(jquery_target) {
        debugger;
            jquery_target = jquery_target || element;
            var style = window.getComputedStyle(jquery_target[0]);
            result = {};
            for (var i=0; i<properties.length; i++) {
                var prop = properties[i];
                result[prop] = style[prop];
            }
            return result
        }
    """, 
    properties=properties)

In [29]:
await get(greeting.element.get_font_properties())

{'font-style': 'normal',
 'font-weight': '700',
 'font-family': 'Verdana, Arial, Helvetica, sans-serif',
 'font-size': '32px'}

In [30]:
await get(greeting.element.get_font_properties(txt.element))

{'font-style': 'normal',
 'font-weight': '400',
 'font-family': 'Verdana, Arial, Helvetica, sans-serif',
 'font-size': '16px'}

In [31]:
from H5Gizmos import Html, do
greeting = Html("<h1>Hello</h1>")
await greeting.show()

window = greeting.window
new_uint = greeting.new(window.Uint8Array, [45, 254, 12, 9])
do(window.console.log(new_uint))

In [32]:
greeting.js_debug()

In [33]:
from H5Gizmos import Html, get
greeting = Html("<h1>Hello</h1>")
await greeting.show()

import numpy as np
A = np.sin(0.01 * np.arange(1000))
reference = greeting.cache("Sines", A)
do(greeting.window.console.log(reference))

In [34]:
link = greeting.my("Sines")
await get(link[3])

0.02999550020249566

In [35]:
greeting.uncache("Sines")
print(await get(greeting.my("Sines")))

None


In [36]:
json_structure = {
    "none": None,
    "number": 3.1,
    "string": "Bohemian Rhapsody",
    "bool": False,
    "dictionary": {"lie": "falsehood"},
    "list": [1,2,"three"],
}

from H5Gizmos import Html, do, get
greeting = Html("<h1>Hello</h1>")
await greeting.show()

reference = await greeting.store_json(json_structure, "my_data")

print("string value", await get(reference["string"]))

string value Bohemian Rhapsody


In [37]:
import numpy as np

A = (np.arange(1000) % 13 - 5).astype(np.float32)

from H5Gizmos import Html, do, get
greeting = Html("<h1>Hello</h1>")
await greeting.show()

reference = await greeting.store_array(A, "my_array")
do(greeting.window.console.log(reference))

In [38]:
B = await greeting.get_array_from_buffer(reference, dtype=np.float32)
np.abs(A - B).max()

0.0

In [39]:
B.shape

(1000,)